In [ ]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

In [ ]:
data=pd.read_csv("forestfires.csv")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isna().sum()#there are no null values.

In [ ]:
data.corr()

In [ ]:
data.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,6))#in the month of augest and september there is fire in forest for small and large areas.
sns.countplot(x = 'month', hue = 'size_category', data = data)

In [ ]:
sns.histplot(x="FFMC",bins=50,hue="size_category",data=data)#wecan see that at higher FFMC we have small and large area fire burned.

In [ ]:
sns.scatterplot(x ='temp',hue ='size_category', data = data)

In [ ]:

sns.countplot(x="rain",hue="size_category",data=data)#shows that if rain properly fire will catch less.

In [ ]:
sns.scatterplot(x="wind",y="size_category",data=data)# show that winds is less effect to catch fire.

In [ ]:
sns.scatterplot(x="area",y="size_category",data=data)# if forest is large that change of catching fire for large area is higher.

In [ ]:
sns.histplot(x="DMC",bins=50,hue="size_category",data=data)

In [ ]:

sns.histplot(x="ISI",bins=50,hue="size_category",data=data)#If ISI content is less than there is large of proposition of catching the fire

In [ ]:
data1=data.iloc[:,0:11]

In [ ]:
data1

In [ ]:
data_nam=data1["month"]
data_num1=data1["day"]

In [ ]:
standard_data=StandardScaler()
data2=standard_data.fit_transform(data1.iloc[:,2:11])
Standard=pd.DataFrame(data2)


In [ ]:
Standard["month"]=data_nam
Standard["day"]=data_num1

In [ ]:
Standard

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

In [ ]:
Standard['month']=label_encoder.fit_transform(data1['month'])
Standard['day']=label_encoder.fit_transform(data1['day'])

In [ ]:
Standard

In [ ]:
Y_=data['size_category']

In [ ]:
Y=LabelEncoder().fit_transform(Y_) # 1 is assigned to small,0 is assigned to large
X=Standard


In [ ]:
#spliting into test and train
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.3,random_state=40)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV,KFold

In [ ]:

def create_model():
  model =Sequential()
  model.add(Dense(10,input_dim=11,activation="relu"))
  model.add(Dense(8,activation="relu"))
  model.add(Dense(8,activation="relu"))

  adam=Adam(lr=0.1)
  model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=adam)
  return model

In [ ]:
model=KerasClassifier(build_fn=create_model)
batch_size=[40,60]
epochs=[50,100]
param_grids=dict(batch_size=batch_size,epochs=epochs)
grid=GridSearchCV(estimator=model,param_grid=param_grids,cv=KFold(),verbose=10)
grid_result=grid.fit(xtrain,ytrain)



In [ ]:
print('best:{},using:{}'.format(grid_result.best_score_,grid_result.best_params_))
mean=grid_result.cv_results_["mean_test_score"]
std=grid_result.cv_results_["std_test_score"]
params=grid_result.cv_results_["params"]
for mean,stdev,param in zip(mean,std,params):
  print('{},{},with: {}'.format(mean, stdev, param))

In [ ]:
#tunning hyperparameter:learning_rate and dropout

In [ ]:
def create_model1(learning_rate,dropout_rate):
  model=Sequential()
  model.add(Dense(10,input_dim=11,activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(8,activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1,activation='relu'))

  adam=Adam(lr=learning_rate)
  model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=adam)
  return model

In [ ]:
from keras.layers import Dropout
model=KerasClassifier(build_fn=create_model1,batch_size=40,epochs=100)
learning_rate=[0.1,0.2,0.01]
dropout_rate=[0.1,0.2,0.15]
param_grids=dict(learning_rate=learning_rate,dropout_rate=dropout_rate)
grid=GridSearchCV(estimator=model,param_grid=param_grids,cv=KFold(),verbose=10)
grid.results=grid.fit(xtrain,ytrain)


In [ ]:
print('best:{},using:{}'.format(grid.results.best_score_,grid.results.best_params_))
mean=grid.results.cv_results_["mean_test_score"]
std=grid.results.cv_results_["std_test_score"]
params=grid.results.cv_results_["params"]
for mean,stdev,param in zip(mean,std,params):
  print('{},{},with: {}'.format(mean, stdev, param))

In [ ]:
#tunning Kernel Initializer,activation_function

In [ ]:
def create_model(init,activation_function):
  model=Sequential()
  model.add(Dense(10,input_dim=11,kernel_initializer=init,activation=activation_function))
  model.add(Dropout(0.1))
  model.add(Dense(8,kernel_initializer=init,activation=activation_function))
  model.add(Dropout(0.2))
  model.add(Dense(1,activation=activation_function))

  adam=Adam(lr=0.2)
  model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=adam)
  return model


In [ ]:
model=KerasClassifier(build_fn=create_model,batch_size=40,epochs=100)

activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

param_grids = dict(activation_function = activation_function,init=init)

grid = GridSearchCV(estimator = model,param_grid =param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(xtrain,ytrain)

In [ ]:
print('best:{},using:{}'.format(grid.result.best_score_,grid.result.best_params_))
mean=grid.result.cv_results_["mean_test_score"]
std=grid.result.cv_results_["std_test_score"]
params=grid.result.cv_results_["params"]
for mean,stdev,param in zip(mean,std,params):
  print('{},{},with: {}'.format(mean, stdev, param))

In [ ]:
#Tuning of Hyperparameter :-Number of Neurons in activation layer

In [ ]:

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 11,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

neuron1 = [4,8,16]
neuron2 = [2,4,8]

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)
grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(xtrain,ytrain)

In [ ]:

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

In [ ]:
#finding the accuacy score by best hyperparamater.
from sklearn.metrics import classification_report, accuracy_score


def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim =11 ,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(4,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001) 
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model


model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)



model.fit(xtrain,ytrain)



y_predict = model.predict(xtest)

# Printing the metrics
print(accuracy_score(ytest,y_predict))

In [ ]:
#accuracy is 77.5